In [1]:
# Test Openai api with langchain is working
from langchain_openai import ChatOpenAI
import os

OpenAI_llm = ChatOpenAI(model="gpt-4o", temperature=0.6, api_key= os.environ['OPENAI_API_KEY'])
messages=[("Say hello if you are working")]
ai_msg = OpenAI_llm.invoke(messages)
ai_msg

AIMessage(content="Hello! I'm here and ready to help. What can I do for you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-3472316d-22a3-496c-8c97-a2fae56a3cc4-0', usage_metadata={'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# Testing agents with langchain
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["serpapi"], llm = OpenAI_llm)
agent = initialize_agent(tools, llm=OpenAI_llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION)

agent.run("What is current value of 1 dollar in rupees?")

C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_9244\1254470714.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm=OpenAI_llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION)
C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_9244\1254470714.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~i

'The current value of 1 US dollar is approximately 86.53 Indian Rupees.'

In [1]:
from langchain_openai import ChatOpenAI
from config import constants

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.6,
    api_key= constants.OPENAI_API_KEY
)

In [2]:
from langchain.chains.conversation.memory import ConversationBufferMemory 
# initialize conversational memory 

conversational_memory = ConversationBufferMemory(
    memory_key='chat_history',
    k=5,
    return_messages = True
)

C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_12580\3732751371.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferMemory(


In [3]:
from langchain.tools import BaseTool
import requests
import json
class GetClientDetails(BaseTool):
    name: str = "Client details extractor"
    description: str = "use this tool when you have given a platform name, idea or platform link and you want to find more information about CEO of platform and proposed idea"
    def _run(self, website):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "sonar-pro",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are artificial intelligence agent which extracts an information by searching about it online"
                        "and returns information strictly in the following structured JSON format if it exists:"
                        "{"
                        "  'Title of website or platform': <title_of_platform>,"
                        "  'Proposed Idea': <proposed_idea>,"
                        "  'CEO Info': <ceo_info>"
                        "}"
                        "Please make sure the response has proper line breaks, no extra spaces, unexpected characters (like \\n, extra quotes), or non-escaped characters."
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "I am giving you platform name,idea or platform link and you have to find latest information about" 
                        "the platform by going to given link and return atleast 200 words description about idea and"
                        "atleast 150 words description about CEO, their goal and achievements"
                        "Please try to make it as detailed as you can and always refer to online information"
                        "here is the platform link or name "
                        f"{str(website)}"
                    )
                },
            ],
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers).json()
        return  response["choices"][0]["message"]["content"]
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

In [4]:
class GetPlatformInfo(BaseTool):
    name: str = "Platform info extractor"
    description: str = "use this tool when you have given a description about meeting and you have to find the proposed idea, client name or platform link from given description"

    def _run(self, description):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "sonar-pro",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You have given a description of meeting and you will have to find out proposed idea, client name and website link mentioned in the given description."
                        "Use client info extractor tool to get more information."
                        "Return it strictly in the following structured JSON format: "
                        "{"
                        "  'Idea': <idea>,"
                        "  'Client Info': <client_info>,"
                        "  'Platform Link': <platform_link>"
                        "}"
                        "Please make sure the response has proper line breaks, no extra spaces, unexpected characters (like \\n, extra quotes), or non-escaped characters."
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "if anything is not provided in description then use internet and google search to find out the actual information"
                        "Here is the meeting description " 
                        f"{str(description)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers).json()
        return response["choices"][0]["message"]["content"]
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

In [5]:
class GetIdeaSolution(BaseTool):
    name: str = "Solution extractor"
    description: str = "use this tool when you have given an idea description and information about client and you have to find solution on how we can achieve the given idea"

    def _run(self, description):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "sonar-pro",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You have given an idea description and you have to find a solution to achieve the given idea using AI-ML"
                        "or webdev technologies. Search online about tech stack, resources, timeline of project and youtube videos and send it with proper formatting strictly in the following structured JSON format with proper line breaks, no extra spaces, unexpected characters (e.g., \\n, extra quotes, unescaped characters), and proper escaping. "
                        "Ensure the response has the following format: "
                        "{"
                        "  'Solution': <solution>,"
                        "  'Tech Stack': <tech_stack>,"
                        "  'Timeline': <timeline>"
                        "}"  
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "Here is the idea description: " 
                        f"{str(description)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers).json()
        return response["choices"][0]["message"]["content"]
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

In [6]:
# Creating Agent
from langchain.agents import initialize_agent
tools = [GetClientDetails(), GetPlatformInfo(), GetIdeaSolution()]
agent = initialize_agent(
    tools=tools, 
    llm = llm, 
    agent="zero-shot-react-description", 
    verbose=True, 
    max_iterations=3, 
    early_stopping_method='generate', 
    memory = conversational_memory, 
    handle_parsing_errors=True
)

C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_12580\1857880921.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [36]:
from openai import OpenAI
import re

def process_description(description:str):
    client = OpenAI(api_key=constants.OPENAI_API_KEY)

    prompt = f"""
        You are an AI assistant that evaluates descriptions of client requests.
        
        A valid description must contain:
        - Information about the IDEA (what the project is about, how it works, or its purpose).
        - Information about the CLIENT (who is the client that has the proposed project idea).
        
        Given the following description, assess whether both criteria are met. 
        If the idea is present, return 'idea_info': True; otherwise, False.
        If the client info is present, return 'client_info': True; otherwise, False.
        Also, return 'valid': True if both are present; otherwise, False.

        Here is the description: {description}

        Provide the result in valid JSON format with keys: 'idea_info', 'client_info', 'valid'. Make sure to use proper line breaks and no extra spaces.
    """
    chat_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that processes and evaluates project description from user"},
            {"role": "user", "content": prompt}
        ]
    )
    result_text = chat_completion.choices[0].message.content

    # Regex pattern to extract the JSON object from the response
    pattern = r"\{[^{}]*\}"

    match = re.search(pattern, result_text, re.DOTALL)  # DOTALL allows matching across multiple lines
    result_json = match.group(0)
    return json.loads(result_json)

# Example Usage
user_input = "An AI-powered chatbot."
result = process_description(user_input)
print(result)

{'idea_info': True, 'client_info': False, 'valid': False}


In [8]:
para = agent.run(
      "First find the idea, client name and platform link using Platform info extractor tool and find 150 words information about proposed idea about product, goals and achievements of CEO. Once you got the information about their idea then find how we can help them to build it in 200 words "
      "fetch realtime data from internet everytime."
      "This information is going to be added in project proposal so please write in detail and sections like CEO_Info,idea and solution must be explained in 600 words each. "
      "Return your response in structured JSON format like below and please use proper line breaks: "
      """
      {
      "platform_name": <platform_name>,
      "CEO_Info": <ceo_info>,
      "CEO_Name": <ceo_name>,
      "idea": <idea>,
      "solution": <solution>,
      "tech_stack": <tech stack>,
      "timeline": <timeline>,
      "Platform_link": <platform_link>
      }
      """
      "Here is the description: "
      "Hey there, myself rohan and we are hosting this meeting to discuss about my idea of making a twitter automation tool with several features like automated tweets, scheduled tweets, tweet improvement guides etc"
      )
print(para)

C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_6564\3114174648.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  para = agent.run(




> Entering new AgentExecutor chain...
To answer this question, I need to follow a series of steps to extract the necessary information and structure it as requested. 

First, I will use the Platform info extractor tool to find the proposed idea, client name, and platform link from the given description.

Action: Platform info extractor
Action Input: "Hey there, myself rohan and we are hosting this meeting to discuss about my idea of making a twitter automation tool with several features like automated tweets, scheduled tweets, tweet improvement guides etc"
Observation: {
  "Idea": "Making a Twitter automation tool with features like automated tweets, scheduled tweets, and tweet improvement guides",
  "Client Info": "Rohan",
  "Platform Link": null
}
Thought:Now that I have the idea and client information, I need to find more information about the CEO (Rohan) and the proposed idea, including the product, goals, and achievements of the CEO. I will use the Client details extractor tool 

In [9]:
from openai import OpenAI
client = OpenAI(api_key = constants.OPENAI_API_KEY)
chat_completion = client.chat.completions.create(
    model="gpt-4o",
    temperature=0.7,
    messages=[
        {
            "role": "user",
            "content": f"""
                I am giving you some data in python dictionary format and you will have to add more information in it and convert it in markdown file with below format
                Don't use anything except headings and paragraphs in markdown 
                # Project name 
                <project_name>
                # Who is client? 
                <information about client>
                # What is the idea? 
                <information about idea>
                # How can we help? 
                <detailed information about solution>
                # Tech stack
                <information about tech stack>
                # Timeline 
                <information about timeline>
                Try to add more information by yourself too to make it more detailed and make it around 800 words. Don't return anything except markdown and use proper line breaks.
                Here is the information about fields in the given dictionary:
                platform_name: Name of platform
                CEO_Info: Background about CEO and information about CEO 
                CEO_Name: Name of CEO
                idea: project idea 
                solution: solution on how we can solve the given problem statement and how to achieve the given idea 
                tech_stack: technology stack used  to build the solution 
                timeline: timeline of the project 
                Platform_link: link to the platform 
                Here is the data:
                {para}
            """
        }
    ]
)
output = chat_completion.choices[0].message.content
print(output)

# Project name 
Twitter Automation Tool

# Who is client? 
The Twitter Automation Tool is a conceptual project designed to serve businesses and individuals who seek to enhance their social media management capabilities. Although there is no specific client or CEO since this is a proposed tool rather than an existing company, the project is rooted in the well-established domain of social media management. Leaders in this space typically have backgrounds in software development, digital marketing, and social media strategy. Their primary objectives include simplifying social media tasks, improving user engagement, and maximizing the online presence of businesses and individuals. Achievements for professionals in this field often involve developing innovative features that distinguish their tools from competitors, securing partnerships with major brands or influencers, and demonstrating measurable improvements in client engagement metrics. Additionally, they focus on staying ahead of Twit

In [10]:
import pypandoc
output = pypandoc.convert_text(output, 'docx', format='md', outputfile="output.docx")